In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
datasets_path = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        datasets_path.append(os.path.join(dirname, filename))
        
# datasets_path
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
# function, input: dataset, model, output: accuracy of the model over the given dataset

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import torchvision
import torchvision.transforms as transforms
from torchvision.utils import make_grid
from sklearn.model_selection import train_test_split


In [7]:
train_dataset = pd.read_csv('/kaggle/input/fashinemnist-color-sampling/fashion-mnist_train_rgb.csv')

train_dataset.shape

(60000, 2353)

In [8]:
# construct the models, models with 2, 3, 4 & 6 layers, 
class MLP2(nn.Module):
    def __init__(self):
        super(MLP2, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(2352, 512),  # First hidden layer with 512 neurons
            nn.ReLU(),
            nn.Linear(512, 256),   # Second hidden layer with 256 neurons
            nn.ReLU(),
            nn.Linear(256, 10)     # Output layer with 10 neurons
        )
        
    def forward(self, x):
        x = self.layers(x)
        return x

class MLP3(nn.Module):
    def __init__(self):
        super(MLP3, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(2352, 1024),  # First hidden layer
            nn.ReLU(),
            nn.Linear(1024, 512),   # Second hidden layer
            nn.ReLU(),
            nn.Linear(512, 256),    # Third hidden layer
            nn.ReLU(),
            nn.Linear(256, 10)      # Output layer
        )
        
    def forward(self, x):
        x = self.layers(x)
        return x

class MLP4(nn.Module):
    def __init__(self):
        super(MLP4, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(2352, 1024),  # First hidden layer
            nn.ReLU(),
            nn.Linear(1024, 512),   # Second hidden layer
            nn.ReLU(),
            nn.Linear(512, 256),    # Third hidden layer
            nn.ReLU(),
            nn.Linear(256, 128),    # Fourth hidden layer
            nn.ReLU(),
            nn.Linear(128, 10)      # Output layer
        )
        
    def forward(self, x):
        x = self.layers(x)
        return x

class MLP6(nn.Module):
    def __init__(self):
        super(MLP6, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(2352, 2048),  # First hidden layer
            nn.ReLU(),
            nn.Linear(2048, 1024),  # Second hidden layer
            nn.ReLU(),
            nn.Linear(1024, 512),   # Third hidden layer
            nn.ReLU(),
            nn.Linear(512, 256),    # Fourth hidden layer
            nn.ReLU(),
            nn.Linear(256, 128),    # Fifth hidden layer
            nn.ReLU(),
            nn.Linear(128, 64),     # Sixth hidden layer
            nn.ReLU(),
            nn.Linear(64, 10)       # Output layer
        )
        
    def forward(self, x):
        x = self.layers(x)
        return x

In [9]:
# define data-loader function
def load_train_data(dataset_path):
    train_df = pd.read_csv(dataset_path)
    print(train_df.shape)
    X_train, X_valid, y_train, y_valid = train_test_split(
        train_df.iloc[:, 1:], train_df['label'], test_size=1/6, random_state=42)
    
    transform_train = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomRotation(10),
        transforms.ToTensor()
    ])
    transform_test = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor()
    ])

    # Convert data to tensors
    X_train_tensor = torch.tensor(X_train.values, dtype=torch.uint8).view(-1, 3,28, 28)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
    X_valid_tensor = torch.tensor(X_valid.values, dtype=torch.uint8).view(-1, 3,28, 28)
    y_valid_tensor = torch.tensor(y_valid.values, dtype=torch.long)

    # Apply transformation
    X_train_tensor = torch.stack([transform_train(img) for img in X_train_tensor])
    X_valid_tensor = torch.stack([transform_test(img) for img in X_valid_tensor])
    
    # Flatten the tensors
    X_train_tensor = X_train_tensor.view(X_train_tensor.size(0), -1)
    X_valid_tensor = X_valid_tensor.view(X_valid_tensor.size(0), -1)

    # Create TensorDatasets
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    valid_dataset = TensorDataset(X_valid_tensor, y_valid_tensor)

    # Create DataLoaders
    train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
    valid_loader = DataLoader(dataset=valid_dataset, batch_size=128, shuffle=False)

    return train_loader

def load_test_data(dataset_path):
    test_df = pd.read_csv(dataset_path)
    X_test = test_df.iloc[:, 1:]
    
    transform_test = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor()
    ])

    X_test_tensor = torch.tensor(X_test.values, dtype=torch.uint8).view(-1, 3, 28, 28)
    X_test_tensor = torch.stack([transform_test(img) for img in X_test_tensor])
    X_test_tensor = X_test_tensor.view(X_test_tensor.size(0), -1)
    test_dataset = TensorDataset(X_test_tensor)
    test_loader = DataLoader(dataset=test_dataset, batch_size=128, shuffle=False)

    return test_loader

In [10]:
def test_class_values(dataset_path):
    test_df = pd.read_csv(dataset_path)
    y_test = test_df.iloc[:, 0]
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

    return y_test_tensor

In [11]:
test_loader = load_test_data('/kaggle/input/fashinemnist-color-sampling/fashion-mnist_test_rgb.csv')
y_test_tensor = test_class_values('/kaggle/input/fashinemnist-color-sampling/fashion-mnist_test_rgb.csv')

In [24]:
# write the train function

def train(data_loader, model, epochs):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.CrossEntropyLoss()
    model.train()
    for epoch in range(epochs):
        for i, (images, labels) in enumerate(train_loader):
        
            optimizer.zero_grad()
            
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()



def test(model, test_loader, y_test_tensor):
    # Set model to evaluation mode
    model.eval()
    test_preds = []

    # Disable gradient calculations
    with torch.no_grad():
        for batch in test_loader:  # Each batch is a list containing one tensor
            images = batch[0].to(next(model.parameters()).device)  # Extract the tensor and move to device

            # Forward pass
            outputs = model(images)

            # Get predictions
            pred = outputs.max(1, keepdim=True)[1]
            test_preds.append(pred.cpu())  # Store predictions on CPU for safety

    # Concatenate all predictions
    test_preds = torch.cat(test_preds).view(-1)
    y_test_tensor = y_test_tensor.view(-1)

    # Calculate accuracy
    correct = (test_preds == y_test_tensor).sum().item()
    total = y_test_tensor.size(0)
    accuracy = 100 * correct / total

    return accuracy


In [ ]:
# run the train function for all (dataset, model) pair
train_dataset_loaders = [load_train_data(path) for path in datasets_path]
models = [MLP2, MLP3, MLP4, MLP6]
result = {}


epoch = 10
for i, train_loader in enumerate(train_dataset_loaders):
    for model in models:
        model = model()
        train(train_loader, model, epoch)
        res = test(model, test_loader, y_test_tensor)

        result[(datasets_path[i], models[i])] = res
    

In [ ]:
for i, train_loader in enumerate(train_dataset_loaders):
    for model in models:
        model = model()
        train(train_loader, model, epoch)
        res = test(model, test_loader, y_test_tensor)

        result[(datasets_path[i], models[i])] = res

In [ ]:
# Analyse and interpret "result"